In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [16]:
fin = pd.read_excel('all-data.xlsx') 
fin

,Sentiment,news
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [17]:
fin.isnull().sum()

Sentiment    0
news         0
dtype: int64

In [18]:
fin.shape

(4846, 2)

In [19]:
fin.Sentiment.value_counts()

neutral     2879
positive    1363
negative     604
Name: Sentiment, dtype: int64

In [24]:
val=fin.Sentiment
val

0        neutral
1        neutral
2       negative
3       positive
4       positive
          ...   
4841    negative
4842     neutral
4843    negative
4844    negative
4845    negative
Name: Sentiment, Length: 4846, dtype: object

In [25]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
labels= le.fit_transform(val)
labels

array([1, 1, 0, ..., 0, 0, 0])

In [55]:
sent= fin.news
sent[2]

'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [28]:
# Tokenize the sentences
tokenizer = Tokenizer()
clean_sentences = [sentence for sentence in sent ]
len(clean_sentences)

4846

In [38]:
tokenizer.fit_on_texts(clean_sentences)
word_index = tokenizer.word_index

In [39]:
word_index

{'the': 1,
 'of': 2,
 'in': 3,
 'and': 4,
 'to': 5,
 'a': 6,
 'for': 7,
 'eur': 8,
 "'s": 9,
 'is': 10,
 'company': 11,
 'will': 12,
 'from': 13,
 'on': 14,
 'its': 15,
 'has': 16,
 'with': 17,
 'by': 18,
 'said': 19,
 'as': 20,
 'be': 21,
 'finnish': 22,
 'mn': 23,
 '1': 24,
 'at': 25,
 'it': 26,
 'sales': 27,
 'million': 28,
 'that': 29,
 'net': 30,
 'profit': 31,
 'year': 32,
 'was': 33,
 'm': 34,
 'finland': 35,
 '2': 36,
 '5': 37,
 '3': 38,
 'an': 39,
 'group': 40,
 '2009': 41,
 'operating': 42,
 '2008': 43,
 '4': 44,
 'mln': 45,
 '0': 46,
 'are': 47,
 'new': 48,
 'business': 49,
 'period': 50,
 'quarter': 51,
 '2007': 52,
 '2010': 53,
 '6': 54,
 'oyj': 55,
 'share': 56,
 '7': 57,
 '8': 58,
 'market': 59,
 '9': 60,
 'also': 61,
 'services': 62,
 '000': 63,
 'which': 64,
 'have': 65,
 "''": 66,
 'shares': 67,
 'first': 68,
 'this': 69,
 'up': 70,
 '2006': 71,
 'helsinki': 72,
 'been': 73,
 'euro': 74,
 'loss': 75,
 'about': 76,
 'compared': 77,
 'today': 78,
 'operations': 79,
 'co

In [40]:
# Convert sentences to sequences of indices
sequences = tokenizer.texts_to_sequences(clean_sentences)

# Pad sequences to ensure they have the same length
maxlen = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

padded_sequences.shape


(4846, 71)

In [41]:
labels.shape

(4846,)

In [43]:
np.unique(labels)

array([0, 1, 2])

# RNN Model

In [44]:
# Define RNN model
embedding_dim = 16
vocab_size = len(word_index) + 1

# Define RNN model for multi-class classification
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    SimpleRNN(32),  # Simple RNN layer with 32 units
    Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, labels, epochs=10, verbose=1)

# Test the model
test_sentence = 'This movie is amazing'
test_sequence = tokenizer.texts_to_sequences([test_sentence])
padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
prediction = model.predict(padded_test_sequence)
predicted_class = np.argmax(prediction)
print("Sentiment prediction:", predicted_class)

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5089 - loss: 1.0831
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6163 - loss: 0.8538
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7788 - loss: 0.5646
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9120 - loss: 0.2876
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9684 - loss: 0.1242
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9877 - loss: 0.0679
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9920 - loss: 0.0413
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9964 - loss: 0.0246
Epoch 9/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9979 - loss: 0.0160
Epoch 10/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9939 - loss: 0.0219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Sentiment prediction: 1


In [48]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Define the prediction function
def predict_sentiment(text):
    # Preprocess the input text
    test_sequence = tokenizer.texts_to_sequences([text])
    padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
    # Predict sentiment using the model
    prediction = model.predict(padded_test_sequence)
    predicted_class = np.argmax(prediction)
    
    # Use LabelEncoder to map predicted class index to original sentiment label
    predicted_label = le.inverse_transform([predicted_class])[0]
    
    return predicted_label

# Create a Gradio interface
iface = gr.Interface(fn=predict_sentiment, 
                     inputs="text",
                     outputs="text",
                     title="Sentiment Analysis of Financial News",
                     description="Enter a text to predict its sentiment (Neutral, Negative or Positive).")
                     
# Launch the interface
iface.launch(share = True)


Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Exception in thread Thread-22 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection.py", line 76, in handle_request
    stream 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


# LSTM Model

In [50]:
from tensorflow.keras.layers import LSTM, Dropout

# Define a more complex LSTM model for multi-class classification
model_complex_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    LSTM(64, return_sequences=True),  # First LSTM layer with 64 units and return sequences
    Dropout(0.5),  # Dropout layer to reduce overfitting
    LSTM(32),  # Second LSTM layer with 32 units
    Dropout(0.5),  # Dropout layer
    Dense(64, activation='relu'),  # Dense layer with ReLU activation
    Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

In [51]:
model_complex_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [52]:
model_complex_lstm.fit(padded_sequences, labels, epochs=10, verbose=1)


Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.5505 - loss: 1.1215
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.6621 - loss: 0.8007
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7649 - loss: 0.5770
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8372 - loss: 0.4158
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8837 - loss: 0.3319
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9052 - loss: 0.2795
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9387 - loss: 0.2173
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9471 - loss: 0.1752
Epoch 9/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9245 - loss: 0.2283
Epoch 10/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9640 - loss: 0.1282


In [54]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Define the prediction function
def predict_sentiment(text):
    # Preprocess the input text
    test_sequence = tokenizer.texts_to_sequences([text])
    padded_test_sequence = pad_sequences(test_sequence, maxlen=maxlen)
    # Predict sentiment using the model
    prediction = model_complex_lstm.predict(padded_test_sequence)
    predicted_class = np.argmax(prediction)
    
    # Use LabelEncoder to map predicted class index to original sentiment label
    predicted_label = le.inverse_transform([predicted_class])[0]
    
    return predicted_label

# Create a Gradio interface
iface = gr.Interface(fn=predict_sentiment, 
                     inputs="text",
                     outputs="text",
                     title="Sentiment Analysis of financial news",
                     description="Enter a text to predict its sentiment (Neutral, negative or positive).")
                     
# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Exception in thread Thread-31 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc
  File "C:\Users\admin\anaconda3\Lib\site-packages\httpcore\_sync\connection.py", line 76, in handle_request
    stream 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
